<a href="https://colab.research.google.com/github/gulsumbudakoglu/glsm/blob/master/Data_Engineer_Mulakat_Projesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns
import scipy as sp
import matplotlib.patches as mpatches
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
from pandas import DataFrame
from numpy.core.defchararray import add
import plotly.graph_objs as go
#import plotly.plotly as py
import plotly.offline as pyoff
import statsmodels.api as sm
import warnings
import matplotlib
import keras

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df1 = pd.read_csv('/content/drive/MyDrive/adjustnewto.csv',sep = ';', error_bad_lines=False, low_memory=False, encoding = "ISO-8859-1")
df1.head()


,{app_id},{app_name},{app_version},{store},{tracker},{tracker_name},{first_tracker},{first_tracker_name},{last_tracker},{last_tracker_name},{network_name},{campaign_name},{adgroup_name},{creative_name},{impression_based},{is_organic},{click_referer},{activity_kind},{click_time},{impression_time},{conversion_duration},{engagement_time},{installed_at},{installed_at_hour},{install_begin_time},{install_finish_time},{referral_time},{created_at},{reattributed_at},{time_to_uninstall},{time_to_reinstall},{uninstalled_at},{reinstalled_at},{last_session_time},{connection_type},{cpu_type},{hardware_name},{network_type},{device_manufacturer},{nonce},...,{time_spent},{session_count},{lifetime_session_count},{is_reattributed},{revenue},{currency},{revenue_usd},{reporting_revenue},{reporting_currency},{cost_type},{cost_amount},{cost_currency},{reporting_cost},{push_token},{gclid},{adwords_campaign_type},{adwords_campaign_name},{adwords_campaign_id},{adwords_adgroup_id},{adwords_creative_id},{adwords_network_type},{adwords_network_subtype},{adwords_keyword},{adwords_matchtype},{adwords_placement},{adwords_video_id},{search_term},{fb_campaign_group_name},{fb_campaign_group_id},{fb_campaign_id},{fb_campaign_name},{fb_adgroup_name},{fb_adgroup_id},{fb_ad_objective_name},{fb_account_id},{fb_platform_position},{iad_creative_set_name},{iad_creative_set_id},{iad_conversion_type},{iad_keywords_matchtype}
0,tr.com.apps.kk,NaN,NaN,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,NaN,NaN,NaN,NaN,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,NaN,NaN,NaN,impression,NaN,1.577977e+09,NaN,1.577977e+09,NaN,NaN,NaN,NaN,NaN,1577977200,NaN,NaN,NaN,NaN,NaN,NaN,Cable/DSL,NaN,NaN,NaN,NaN,dddfet91v,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{iad_keywords_matchtype}
1,tr.com.apps.kk,NaN,NaN,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,NaN,NaN,NaN,NaN,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,NaN,NaN,NaN,impression,NaN,1.577977e+09,NaN,1.577977e+09,NaN,NaN,NaN,NaN,NaN,1577977200,NaN,NaN,NaN,NaN,NaN,NaN,Cellular,NaN,NaN,NaN,NaN,17jk8430o,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{iad_keywords_matchtype}
2,NaN,NaN,NaN,NaN,j9w70pz,AppLovin Installs::KK - Android - TR - AppLovi...,NaN,NaN,NaN,NaN,AppLovin Installs,KK - Android - TR - AppLovin,0dd1c5d826388f1ecb8c75854e406882,Banner_KK - Android - TR - AppLovin,NaN,NaN,NaN,impression,NaN,1.577977e+09,NaN,1.577977e+09,NaN,NaN,NaN,NaN,NaN,1577977200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5zpnyeryd,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{iad_keywords_matchtype}
3,tr.com.apps.kk,NaN,NaN,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,NaN,NaN,NaN,NaN,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,NaN,NaN,NaN,impression,NaN,1.577977e+09,NaN,1.577977e+09,NaN,NaN,NaN,NaN,NaN,1577977199,NaN,NaN,NaN,NaN,NaN,NaN,Cable/DSL,NaN,NaN,NaN,NaN,44d9g6brn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{iad_keywords_matchtype}
4,tr.com.apps.kk,NaN,NaN,google,gl7y3ir,ironSrc Installs::Kelime Kutusu - Android - TR...,NaN,NaN,NaN,NaN,ironSrc Installs,Kelime Kutusu - Android - TR - ironSrc_8310429,209799,Set-7_122525,NaN,NaN,NaN,impression,NaN,1.577977e+09,NaN,1.577977e+09,NaN,Na

In [4]:
#df1.describe()
#df1.dtypes
df1.shape

(39139, 98)

In [5]:
# see how many there are missing elements in each columns.
df1.isna().sum()

{app_id}                     8091
{app_name}                  34578
{app_version}               34578
{store}                      8091
{tracker}                       0
                            ...  
{fb_platform_position}      38820
{iad_creative_set_name}     39139
{iad_creative_set_id}       39139
{iad_conversion_type}       39139
{iad_keywords_matchtype}        0
Length: 98, dtype: int64

In [6]:
#notice that there are some columns which all the values are null. Delete this columns.
df1 = df1.dropna(how='all', axis=1)
#delete the values are null in rows if we have
df1 = df1.dropna(how='all')


In [7]:
df1.dtypes
df1.nunique()
#df1.dtypes


{app_id}                       2
{app_name}                     1
{app_version}                 20
{store}                        2
{tracker}                   1866
                            ... 
{fb_adgroup_id}                7
{fb_ad_objective_name}         1
{fb_account_id}                1
{fb_platform_position}         2
{iad_keywords_matchtype}       1
Length: 76, dtype: int64

In [8]:
df1.isna().any()

{app_id}                     True
{app_name}                   True
{app_version}                True
{store}                      True
{tracker}                   False
                            ...  
{fb_adgroup_id}              True
{fb_ad_objective_name}       True
{fb_account_id}              True
{fb_platform_position}       True
{iad_keywords_matchtype}    False
Length: 76, dtype: bool

In [9]:
#print (df1.isnull().mean()) #check the percentage of null elements.

#If more than %80 of NAN values is in one column and i have no other way to complete this i delete this columns.
df = df1.loc[:, df1.isin([' ','NULL',0, 'NaN']).mean() < .8] 
df = df1.loc[:, df1.isnull().mean() <= .8]
#df = df.dropna(subset=['{app_id}'])    #id cannot be NaN thus Drop NaN app_id
print (df.isnull().mean())

{app_id}                    0.206725
{store}                     0.206725
{tracker}                   0.000000
{tracker_name}              0.000000
{network_name}              0.000000
{campaign_name}             0.039730
{adgroup_name}              0.039781
{creative_name}             0.048366
{activity_kind}             0.000000
{impression_time}           0.156928
{engagement_time}           0.039730
{created_at}                0.000000
{connection_type}           0.208437
{nonce}                     0.000000
{ip_address}                0.206367
{user_agent}                0.000000
{region}                    0.676512
{country}                   0.000000
{city}                      0.246174
{device_name}               0.242086
{device_type}               0.000588
{os_name}                   0.000000
{os_version}                0.207031
{cost_type}                 0.724674
{cost_amount}               0.724674
{cost_currency}             0.724674
{reporting_cost}            0.724725
{

In [10]:
df.head()
df.shape
#df.dtypes

(39139, 28)

In [11]:
df.head()


,{app_id},{store},{tracker},{tracker_name},{network_name},{campaign_name},{adgroup_name},{creative_name},{activity_kind},{impression_time},{engagement_time},{created_at},{connection_type},{nonce},{ip_address},{user_agent},{region},{country},{city},{device_name},{device_type},{os_name},{os_version},{cost_type},{cost_amount},{cost_currency},{reporting_cost},{iad_keywords_matchtype}
0,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,1.577977e+09,1.577977e+09,1577977200,Cable/DSL,dddfet91v,176.88.209.171,Mozilla/5.0 (Linux; Android 9; SM-A205F Build/...,NaN,tr,Kilis,GalaxyA20,phone,android,9,NaN,NaN,NaN,NaN,{iad_keywords_matchtype}
1,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,1.577977e+09,1.577977e+09,1577977200,Cellular,17jk8430o,37.155.170.239,Mozilla/5.0 (Linux; Android 8.1.0; SM-J260F Bu...,NaN,tr,NaN,GalaxyJ2Core,phone,android,8.1.0,NaN,NaN,NaN,NaN,{iad_keywords_matchtype}
2,NaN,NaN,j9w70pz,AppLovin Installs::KK - Android - TR - AppLovi...,AppLovin Installs,KK - Android - TR - AppLovin,0dd1c5d826388f1ecb8c75854e406882,Banner_KK - Android - TR - AppLovin,impression,1.577977e+09,1.577977e+09,1577977200,NaN,5zpnyeryd,NaN,Mozilla/5.0 (Linux; Android 7.0; Lenovo K53a48...,tr,tr,NaN,NaN,phone,server,NaN,NaN,NaN,NaN,NaN,{iad_keywords_matchtype}
3,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,1.577977e+09,1.577977e+09,1577977199,Cable/DSL,44d9g6brn,81.213.87.36,Mozilla/5.0 (Linux; Android 6.0.1; SM-G5510 Bu...,NaN,tr,Konya,GalaxyOn,phone,android,6.0.1,NaN,NaN,NaN,NaN,{iad_keywords_matchtype}
4,tr.com.apps.kk,google,gl7y3ir,ironSrc Installs::Kelime Kutusu - Android - TR...,ironSrc Installs,Kelime Kutusu - Android - TR - ironSrc_8310429,209799,Set-7_122525,impression,1.577977e+09,1.577977e+09,1577977200,Cable/DSL,x7kx2t35w,88.230.105.60,finagle-http,NaN,tr,Bursa,NaN,phone,android,5.0.2,CPI,"0,07",USD,"0,07",{iad_keywords_matchtype}


In [12]:
Engagement_Data = df[['{app_id}','{store}','{tracker}', '{tracker_name}','{network_name}','{campaign_name}','{adgroup_name}','{creative_name}', '{activity_kind}','{impression_time}','{engagement_time}','{created_at}','{connection_type}']]

In [13]:
Engagement_Data.head()
#Engagement_Data.shape
Engagement_Data.isna().sum()

{app_id}             8091
{store}              8091
{tracker}               0
{tracker_name}          0
{network_name}          0
{campaign_name}      1555
{adgroup_name}       1557
{creative_name}      1893
{activity_kind}         0
{impression_time}    6142
{engagement_time}    1555
{created_at}            0
{connection_type}    8158
dtype: int64

In [74]:
User_Data = df[['{ip_address}','{user_agent}','{region}','{country}','{city}','{device_name}','{device_type}','{os_name}','{os_version}','{cost_type}','{cost_amount}','{cost_currency}','{reporting_cost}']]

In [75]:
User_Data.head()
User_Data.isna().sum() 


{ip_address}         8077
{user_agent}            0
{region}            26478
{country}               0
{city}               9635
{device_name}        9475
{device_type}          23
{os_name}               0
{os_version}         8103
{cost_type}         28363
{cost_amount}       28363
{cost_currency}     28363
{reporting_cost}    28365
dtype: int64

In [16]:
Engagement_Data.head()
Engagement_Data.nunique()

{app_id}                2
{store}                 2
{tracker}            1866
{tracker_name}       1866
{network_name}         13
{campaign_name}        31
{adgroup_name}       1379
{creative_name}       114
{activity_kind}         7
{impression_time}    3624
{engagement_time}    5333
{created_at}         4907
{connection_type}       3
dtype: int64

In [17]:
Engagement_Data.groupby(['{app_id}']).sum()
Engagement_Data['{app_id}'].value_counts()
#Engagement_Data[['{app_id}', '{store}']] = Engagement_Data[['{app_id}', '{store}']].fillna(method='ffill', inplace=True)
Engagement_Data.fillna(method='ffill', inplace=True)
#Engagement_Data.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
Engagement_Data['{impression_time}'] = Engagement_Data['{impression_time}'].astype(np.int64)

Engagement_Data['{engagement_time}'] = Engagement_Data['{engagement_time}'].astype(np.int64)

Engagement_Data['{impression_time}']= pd.to_datetime(Engagement_Data['{impression_time}'], unit='s')

#Engagement_Data['{engagement_time}']= pd.to_datetime(Engagement_Data['{engagement_time}'], unit='s')
#Engagement_Data['{created_at}']= pd.to_datetime(Engagement_Data['{created_at}'], unit='s')



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [19]:
Engagement_Data['{engagement_time}']= pd.to_datetime(Engagement_Data['{engagement_time}'], unit='s')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
Engagement_Data['{created_at}']= pd.to_datetime(Engagement_Data['{created_at}'], unit='s')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
Engagement_Data.head()

,{app_id},{store},{tracker},{tracker_name},{network_name},{campaign_name},{adgroup_name},{creative_name},{activity_kind},{impression_time},{engagement_time},{created_at},{connection_type}
0,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,2020-01-02 15:00:00,2020-01-02 15:00:00,2020-01-02 15:00:00,Cable/DSL
1,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,2020-01-02 15:00:00,2020-01-02 15:00:00,2020-01-02 15:00:00,Cellular
2,tr.com.apps.kk,google,j9w70pz,AppLovin Installs::KK - Android - TR - AppLovi...,AppLovin Installs,KK - Android - TR - AppLovin,0dd1c5d826388f1ecb8c75854e406882,Banner_KK - Android - TR - AppLovin,impression,2020-01-02 15:00:00,2020-01-02 15:00:00,2020-01-02 15:00:00,Cellular
3,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,2020-01-02 14:59:59,2020-01-02 14:59:59,2020-01-02 14:59:59,Cable/DSL
4,tr.com.apps.kk,google,gl7y3ir,ironSrc Installs::Kelime Kutusu - Android - TR...,ironSrc Installs,Kelime Kutusu - Android - TR - ironSrc_8310429,209799,Set-7_122525,impression,2020-01-02 15:00:00,2020-01-02 15:00:00,2020-01-02 15:00:00,Cable/DSL


In [22]:
Engagement_Data['new_{created_at}_date'] = [d.date() for d in Engagement_Data['{created_at}']]
Engagement_Data['new_{created_at}_time'] = [d.time() for d in Engagement_Data['{created_at}']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
Engagement_Data1 = Engagement_Data.drop(['{impression_time}', '{engagement_time}','{created_at}' ], axis=1)


In [24]:
Engagement_Data1.dtypes

{app_id}                 object
{store}                  object
{tracker}                object
{tracker_name}           object
{network_name}           object
{campaign_name}          object
{adgroup_name}           object
{creative_name}          object
{activity_kind}          object
{connection_type}        object
new_{created_at}_date    object
new_{created_at}_time    object
dtype: object

In [25]:

#Engagement_Data1.assign(id=add(np.arange(1, len(df) + 1).astype(str)))
Engagement_Data1["id"] = Engagement_Data1.index + 1

In [26]:
Engagement_Data1.head()

,{app_id},{store},{tracker},{tracker_name},{network_name},{campaign_name},{adgroup_name},{creative_name},{activity_kind},{connection_type},new_{created_at}_date,new_{created_at}_time,id
0,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,Cable/DSL,2020-01-02,15:00:00,1
1,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,Cellular,2020-01-02,15:00:00,2
2,tr.com.apps.kk,google,j9w70pz,AppLovin Installs::KK - Android - TR - AppLovi...,AppLovin Installs,KK - Android - TR - AppLovin,0dd1c5d826388f1ecb8c75854e406882,Banner_KK - Android - TR - AppLovin,impression,Cellular,2020-01-02,15:00:00,3
3,tr.com.apps.kk,google,ptteq5u,TikTok Android Installs::Apps_Kelime Kutusu_CP...,TikTok Android Installs,Apps_Kelime Kutusu_CPI&1645632596191286,Apps_ Kelime Kutusu_Android_Pizza&165370631682...,Kelime Bulmaca_Android_21s_Pizza&1653706321611782,impression,Cable/DSL,2020-01-02,14:59:59,4
4,tr.com.apps.kk,google,gl7y3ir,ironSrc Installs::Kelime Kutusu - Android - TR...,ironSrc Installs,Kelime Kutusu - Android - TR - ironSrc_8310429,209799,Set-7_122525,impression,Cable/DSL,2020-01-02,15:00:00,5


In [ ]:
Engagement_Data1.to_excel("Engagement_Data.xlsx") 

In [76]:

User_Data.nunique()

{ip_address}        25418
{user_agent}         3835
{region}               20
{country}              26
{city}                264
{device_name}         786
{device_type}           5
{os_name}               4
{os_version}           75
{cost_type}             1
{cost_amount}          19
{cost_currency}         1
{reporting_cost}       19
dtype: int64

In [77]:
#we have country thus drop 'region' and 'ip_address'
#we have just one type 'cost_type' and 'cost_currency' thus drop these.
#'reporting cost' and cost amount have same values and thus drop reporting cost
User_Data = User_Data.drop(['{region}', '{ip_address}', '{cost_type}', '{cost_currency}', '{reporting_cost}'], axis=1)





In [78]:
#User_Data['{city}'] = User_Data['{city}'].mask(User_Data.{country} == 1, 0)
#User_Data['{city}'] = np.where(User_Data['{city}'] == 'NaN', User_Data['{country}'], User_Data['{city}'])
User_Data['{city}'].fillna(User_Data['{country}'], inplace=True)

In [80]:
User_Data['{device_type}'].fillna(method='ffill', inplace=True)

In [81]:
User_Data['{device_name}'].fillna(User_Data['{device_type}'], inplace=True)

In [82]:
User_Data.dtypes

{user_agent}     object
{country}        object
{city}           object
{device_name}    object
{device_type}    object
{os_name}        object
{os_version}     object
{cost_amount}    object
dtype: object

In [83]:
User_Data['{cost_amount}'].describe()
User_Data['{cost_amount}'].value_counts()

0,05    7330
0,06    1586
0,13     693
0,04     422
0,07     226
0,15     147
0,08      86
0,18      83
0,11      55
0,02      40
0,03      35
0,12      25
0,14      16
0,17      13
0,1        6
0,09       5
0,38       3
0,16       3
0,68       2
Name: {cost_amount}, dtype: int64

In [84]:
#see that  in 'cost_amount', the most comman value is 0,05 so fill all NaN with 0,05
User_Data['{cost_amount}'] = User_Data['{cost_amount}'].fillna(User_Data['{cost_amount}'].value_counts().index[0])

In [86]:
User_Data["id"] = User_Data.index + 1

In [87]:
User_Data.head()

,{user_agent},{country},{city},{device_name},{device_type},{os_name},{os_version},{cost_amount},id
0,Mozilla/5.0 (Linux; Android 9; SM-A205F Build/...,tr,Kilis,GalaxyA20,phone,android,9,"0,05",1
1,Mozilla/5.0 (Linux; Android 8.1.0; SM-J260F Bu...,tr,tr,GalaxyJ2Core,phone,android,8.1.0,"0,05",2
2,Mozilla/5.0 (Linux; Android 7.0; Lenovo K53a48...,tr,tr,phone,phone,server,NaN,"0,05",3
3,Mozilla/5.0 (Linux; Android 6.0.1; SM-G5510 Bu...,tr,Konya,GalaxyOn,phone,android,6.0.1,"0,05",4
4,finagle-http,tr,Bursa,phone,phone,android,5.0.2,"0,07",5


In [88]:
User_Data.to_excel("User_Data.xlsx")